In [4]:
import cv2
import os
import numpy as np
import math

In [5]:
def read_images(img_dir):

    # Define the image file extensions to be read
    img_exts = (".jpg", ".jpeg", ".png")

    # Initialize an empty list to store the images
    images = []

    # Loop through all the files in the directory
    for file in sorted(os.listdir(img_dir)):
        # Check if the file has a valid image extension
        if file.lower().endswith(img_exts):
            # Read the image using OpenCV
            img = cv2.imread(os.path.join(img_dir, file))
            # # Convert the image to grayscale and append it to the list
            # gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            images.append(img)
            

    # Convert the list of images to a 2D numpy array
    img_array = np.array(images)

    return img_array

In [6]:
def cylindrical_warp(img_array, f):

    proj_images = []

    for img in img_array:
        # Get the dimensions of each image
        # height, width, channels = img.shape
        height, width, channel = img.shape
        # Define the center of the cylindrical coordinates
        x0 = width // 2
        y0 = height // 2

        

        # Create a new image with the same size as the original image
        proj_img = np.zeros_like(img)
        # Iterate over each pixel in the new image
        for i in range(height):
            for j in range(width):
                b=img[i,j,0]
                g=img[i,j,1]
                r=img[i,j,2]
                # Convert the pixel's (x, y) coordinates to cylindrical coordinates (h, theta)
                x = j - x0
                y = i - y0

                h = y/math.sqrt(x**2 + f**2)
                theta = math.atan(x / f)

                # Calculate the new (x, y) coordinates from the cylindrical coordinates using the inverse transform
                
                x_new = f*theta + x0
                y_new = f*h + y0

                # Round the new (x, y) coordinates to the nearest integer to get the corresponding pixel in the original image
                x_new_rd = int(round(x_new))
                y_new_rd = int(round(y_new))

                proj_img[y_new_rd, x_new_rd, 0] = b
                proj_img[y_new_rd, x_new_rd, 1] = g
                proj_img[y_new_rd, x_new_rd, 2] = r


        proj_images.append(proj_img)

    proj_array = np.array(proj_images)

    # cv.imshow("proj_img0", proj_array[0])
    # cv.waitKey(0)
    # cv.destroyAllWindows()

    return proj_array

In [7]:
def feature_detection(proj_array):

    # Read in the image
    img = proj_array[0]

    # Convert the image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Compute the derivatives of the image intensity in the x and y directions
    dx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
    dy = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)

    # Compute the element-wise product of the derivative images
    Ix2 = dx * dx
    Iy2 = dy * dy
    Ixy = dx * dy

    # Apply a Gaussian filter to each of the derivative images
    ksize = (5, 5)
    sigma = 1.0
    Ix2 = cv2.GaussianBlur(Ix2, ksize, sigma)
    Iy2 = cv2.GaussianBlur(Iy2, ksize, sigma)
    Ixy = cv2.GaussianBlur(Ixy, ksize, sigma)

    # Compute the sums of the products of derivatives at each pixel using a sliding window
    w_size = 5
    offset = w_size // 2
    height, width = gray.shape
    Sxx = np.zeros_like(gray, dtype=np.float32)
    Syy = np.zeros_like(gray, dtype=np.float32)
    Sxy = np.zeros_like(gray, dtype=np.float32)

    for y in range(offset, height - offset):
        for x in range(offset, width - offset):
            Sxx[y, x] = np.sum(Ix2[y - offset:y + offset + 1, x - offset:x + offset + 1])
            Syy[y, x] = np.sum(Iy2[y - offset:y + offset + 1, x - offset:x + offset + 1])
            Sxy[y, x] = np.sum(Ixy[y - offset:y + offset + 1, x - offset:x + offset + 1])

    # Compute the determinant and trace of the structure tensor
    det = (Sxx * Syy) - (Sxy ** 2)
    trace = Sxx + Syy

    # Compute the corner response function
    k = 0.04
    response = det - k * (trace ** 2)

    # Threshold the corner response function
    threshold = 0.01 * response.max()
    corners = np.argwhere(response > threshold)

    # Draw crosses at the detected corners on the image
    for corner in corners:
        x, y = corner
        size = 1
        color = (0, 0, 255)
        cv2.line(img, (y - size, x), (y + size, x), color, 2)
        cv2.line(img, (y, x - size), (y, x + size), color, 2)

    return img

In [8]:
if __name__ == '__main__':
    img_array = read_images("test_images")
    proj_array = cylindrical_warp(img_array, 900)
    img = feature_detection(proj_array)

    # Show the image with detected corners
    cv2.imshow("Corners Detected", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()